In [128]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.special import expit
from scipy.optimize import fmin, minimize
import glob as glob
import matplotlib
matplotlib.style.use('ggplot')

# .py
%load_ext autoreload
%autoreload 2
import weather_alphas, weather_costs, model, model_01, validate, prediction

getting bounded alphas ...
done getting alphas ...
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:
def split_data(data):
    train_mask = (data.index < "2011")
    train = data[train_mask]
    test_mask = (data.index > "2011")
    test = data[test_mask]
    return train, test

In [130]:
all_weather = pd.read_csv("weather01-16.csv")

In [131]:
cm_weather = all_weather[all_weather.stn_name == "Chiang Mai"]
cm_weather.index = pd.DatetimeIndex(cm_weather.date)
mask = (cm_weather.index > "2003") & (cm_weather.index < "2016")
cm_weather = cm_weather[mask]

provinces_df = pd.read_csv("all-dengues.csv")
provinces_df.index = pd.DatetimeIndex(provinces_df.date)
provinces_df = provinces_df.drop(['date','date.1'],axis=1)
cm_dengues = provinces_df[provinces_df['จังหวัด'] == 'ChiangMai'].resample('W').size()
cm_dengues_df = pd.DataFrame(cm_dengues,columns=['cases'])

In [132]:
cm_avg_weather = cm_weather[['avgrh','dday','meantemp']].resample('W').mean()
cm_avg_weather['rain'] = cm_weather[['rain']].resample('W').sum() # cumulative rainfall
cm_dengues_weather = pd.concat([cm_avg_weather,cm_dengues_df[:-52]],axis=1)

cm_dengues_weather_split = split_data(cm_dengues_weather)
cm_dengues_train, cm_dengues_test = cm_dengues_weather_split[0], cm_dengues_weather_split[1]

In [133]:
def make_line(start_week,real,predictions,title,x_axis,y_axis,
              real_legend='Real',predict_legend='Prediction'):
    plt.figure(figsize=(12,6))
    plt.plot(range(len(real)),real,'-r',label=real_legend)
    plt.plot(range(start_week,len(predictions)+start_week),predictions,'-k',label=predict_legend,alpha=0.4)
    plt.axvline(start_week)
    plt.text((start_week*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.legend()
    plt.title(title)
    
def plot_all_predictions(LAG, real, province, nweeks_to_predict, all_predictions):
    
    plt.figure(figsize=(13,6))
    plt.plot(range(len(real)),real,label='real')
    graph_title = "%s's Dengue Prediction with Mean Temperature"%province
    plt.title(graph_title)
    for i in range(len(all_predictions)):
        cur_week_guess = nweeks_to_predict[i]
        start_predict_week = LAG+(cur_week_guess-1)
        predicted_cases = all_predictions[i]
        x_weeks = range(start_predict_week,len(predicted_cases)+start_predict_week)
        line_label = "%d week guess"%cur_week_guess
        plt.plot(x_weeks,predicted_cases,label=line_label,alpha=0.8)
    
    seperate_train_test(cm_dengues_df.cases,cm_dengues_train)
    plt.axvline(LAG)
    plt.text((LAG*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel("# week starting 2003")
    plt.ylabel("Dengue cases")
    plt.legend()
    plt.savefig("16-lags-16-weeks-9-temp-wks-chiangmai-00.png")
    
def seperate_train_test(all_data,train_data):
    plt.axvspan(0, len(train_data), color='blue', alpha=0.1)
    plt.text((len(train_data)/2)-25,700,'Train Period',size=16)
    plt.axvline(len(train_data)+1,linewidth=0.5, color='green')
    plt.text(len(train_data)+20,700,'Test Period',size=16)
    plt.axvspan(len(train_data)+1, len(all_data), color='green', alpha=0.1)

In [151]:
import multiprocessing as mp
from multiprocessing import Process, Queue

# Define an output queue
output = Queue()

# def validate

# def compute_ret(temp):
#     for i in range(int(1e5)): pass
#     return temp/2.


ws_csv = np.array(pd.read_csv("best-for-18-lags-ws.csv").T)[0]
    
def validation(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, train, real, output):
                                    # get_alphas takes 4 args, if theres no input csv
    alphas = weather_alphas.get_alphas(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, train, ws_csv)
                                    # get_predictions takes 6 args
    predictions_to_validate = prediction.get_predictions(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, real, alphas.x, "CM")
                                    # get_validations takes 3 argsg
    validated_result = validate.get_validations(LAG, predictions_to_validate, real)
    output.put((LAG,validated_result,alphas.x)) # add in ws as well alphas.x


In [152]:
%%time
# Setup a list of processes that we want to run
train = cm_dengues_train[:52]
real = cm_dengues_weather
processes = [Process(target=validation, args=(weeks, 9, 10, train, real, output)) for weeks in range(15,25)]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

39
bnds 39
40
bnds 40
ws_csv
prev_ws 39
41
ws_csv
bnds 41
prev_ws 40
ws_csv
42
prev_ws 41
43
bnds 42
ws_csv
bnds 43
44
ws_csv
prev_ws 43
bnds 44
prev_ws 42
ws_csv
prev_ws 44
46
45
bnds 45
ws_csv
48
prev_ws 45
47
bnds 47
bnds 48
ws_csv
prev_ws 47
bnds 46
ws_csv
prev_ws 48
ws_csv
prev_ws 46


Process Process-94:
Process Process-96:
Process Process-100:
Traceback (most recent call last):
Process Process-97:
Process Process-93:
Traceback (most recent call last):
Process Process-91:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-92:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-95:
Process Process-98:
Process Process-99:
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Traceback (most recent call last):
  File "/usr/lib

  File "/home/senior-project/.local/lib/python2.7/site-packages/scipy/optimize/lbfgsb.py", line 328, in _minimize_lbfgsb
  File "/home/senior-project/.local/lib/python2.7/site-packages/scipy/optimize/lbfgsb.py", line 328, in _minimize_lbfgsb
  File "/home/senior-project/.local/lib/python2.7/site-packages/scipy/optimize/_minimize.py", line 450, in minimize
  File "/home/senior-project/.local/lib/python2.7/site-packages/scipy/optimize/lbfgsb.py", line 274, in func_and_grad
    callback=callback, **options)
  File "/home/senior-project/.local/lib/python2.7/site-packages/scipy/optimize/optimize.py", line 628, in _approx_fprime_helper
    g = _approx_fprime_helper(x, fun, epsilon, args=args, f0=f)
  File "/home/senior-project/.local/lib/python2.7/site-packages/scipy/optimize/lbfgsb.py", line 274, in func_and_grad
    g = _approx_fprime_helper(x, fun, epsilon, args=args, f0=f)
    g = _approx_fprime_helper(x, fun, epsilon, args=args, f0=f)
    grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
 

    return function(*(wrapper_args + args))
    all_temps = np.dot(weights,pad_y)
  File "weather_alphas.py", line 36, in cost
  File "/home/senior-project/.local/lib/python2.7/site-packages/pandas/core/series.py", line 255, in __init__
    rains_for_prediction)
    self._set_axis(0, index, fastpath=True)
    temperature_part = temperature_term(w_temp, temps_for_prediction) # takes the third part of w
KeyboardInterrupt
  File "/home/senior-project/.local/lib/python2.7/site-packages/pandas/core/series.py", line 284, in _set_axis
  File "weather_costs.py", line 31, in nweek_ahead_cost
  File "model_01.py", line 34, in temperature_term
    def _set_axis(self, axis, labels, fastpath=False):
  File "weather_costs.py", line 20, in nweek_ahead_cost
  File "weather_costs.py", line 31, in nweek_ahead_cost
    pad_y = np.concatenate(([1], np.array(temps_for_prediction)))
    real_case = data.cases[aweek+end_cases_week] +1 # avoid getting zeros
    rains_for_prediction)
  File "/home/senior-proje

KeyboardInterrupt: 

  File "/home/senior-project/.local/lib/python2.7/site-packages/pandas/core/internals.py", line 4226, in get_values
    return np.array(self._block.to_dense(), copy=False)
  File "/home/senior-project/.local/lib/python2.7/site-packages/pandas/core/internals.py", line 169, in to_dense
    def to_dense(self):
KeyboardInterrupt


In [54]:
def get_best(validations):
    idx = None
    best_so_far = validations[0][1][-1]
    print best_so_far
    for i in range(len(validations)):
        cur_validation = validations[i][1]
#         print cur_validation
        if cur_validation[0] <= best_so_far:
            best_so_far = cur_validation[0]
            idx = i
    return validations[idx]
        
    
best_param = get_best(results)

4748.1532393


In [147]:
# pd.DataFrame(best_param[2]).to_csv("best-for-18-lags-ws.csv",index=False)

lg = [16,17,18,20,22]
a = np.array(pd.read_csv("best-for-18-lags-ws.csv").T)[0]

# for i in lg:
#     if i == len(a):
#         arr = a
#         print len(arr)
#     elif i < len(a):
#         arr = a[:-(18-i)]
#         print len(arr)
#     else:
#         mean_val = 1/float(i)
#         arr = np.append(a,[mean_val]*(i-len(a)))
#         print arr
        
def ws_helper(LAG, ws, 18):
    n_ws = len(ws)
    if LAG == n_ws:
        print "eq"
        arr = ws[:LAG+1]
    elif LAG < n_ws:
        print "les"
        arr = ws[:-(n_ws - LAG + 1)]
    else:
        print "else"
        tail = (LAG - n_ws) - 1
        mean_val = 1/(float(LAG)*tail)
        arr = np.append(ws,[mean_val]*tail)
    return arr

len(ws_helper(20,a))

les


19

In [69]:
%%time
[compute_ret(temp.get_alphas(i,9,10)) for i in range(16,26)]

NameError: name 'compute_ret' is not defined